In [2]:
library(edgeR)
library(sva)
library(tidyr)
source('../../ABCA7lof2//degs.r')
set.seed(5)

In [3]:
all_data = readRDS('../../processed_data/ex_layer_stats_input.rds')

In [4]:
summed_counts = all_data$summed_counts_by_ind 
meta = all_data$summary
meta$seq_batch = ifelse(meta$seq_batch=='JBM',1,0)
expressed = all_data$expressed10

In [5]:
# filter out individuals with too few cells per celltype
keep = summed_counts$ncells>=10
summed_counts_indexed = summed_counts$summed_counts[,keep]

In [6]:
if(sum(keep==FALSE)==0){
    print('NOT removing any samples')
}else{
    print(paste0('removing ',str( sum(keep==FALSE)), ' samples'))
}

 int 4
[1] "removing  samples"


In [13]:
# compute degs (all samples)
vars = c('LOF', 'amyloid', 'nft',  'msex', 'age_death', 'pmi', 'seq_batch', 'APOE4')
limma_inputs = get_limma_inputs(summed_counts_indexed, expressed, meta, vars)
aggs = limma_inputs$aggs
metadata = limma_inputs$metadata


degs_all = lapply(names(aggs)[!names(aggs)%in%c('Vascular')], function(x) RunDiffExprAnalysisLimma(aggs[[x]], metadata[[x]], n.sv=NULL))
names(degs_all) = names(aggs)[!names(aggs)%in%c('Vascular')]

Number of significant surrogate variables is:  8 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  8 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  7 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  6 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  6 
Iteration (out of 5 ):1  2  3  4  5  

In [16]:
# save the results
degs = list()
degs[['degs_all']] = degs_all

saveRDS(degs, '../../processed_data/pseudobulk_degs_ExLayers.rds')